In [14]:
import pandas as pd
import datetime as dt
import numpy as np


from pyspark.sql.functions import mean, min, max, sum, datediff, to_date
from pyspark.sql.functions import to_utc_timestamp, unix_timestamp, lit, datediff, col

from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark RFM example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

sc = spark.sparkContext



In [20]:

flat_data_rdd = spark.read.format('com.databricks.spark.csv')   \
        .options(header='true',  inferschema='true') \
        .load('../ressources/data/financial_sells_1000000.csv',header=True);

flat_data_rdd.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- company_name: string (nullable = true)
 |-- company_id: integer (nullable = true)
 |-- country: string (nullable = true)
 |-- sector_name: string (nullable = true)
 |-- order_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- NBI: double (nullable = true)
 |-- order_date: timestamp (nullable = true)



In [21]:
date_max = flat_data_rdd.select(max('order_date')).toPandas()
current = to_utc_timestamp( unix_timestamp(lit(str(date_max.iloc[0][0])), 'yy-MM-dd').cast('timestamp'), 'UTC' )

# Calculatre Duration
flat_data_rdd = flat_data_rdd.withColumn('duration', datediff(lit(current), 'order_date'))

In [38]:
current

Column<b'to_utc_timestamp(CAST(unix_timestamp(2019-07-31 00:00:00, yy-MM-dd) AS TIMESTAMP), UTC)'>